# HW5: Unsupervised Speech Recognition (USR)

Welcome to HW5 in Introduction to Deep Learning 11685. You will be working on Unsupervised Speech Recognition with GANs in this HW. You will be reimplementing and further improving on the model given in the USR paper by Facebook AI.<br>
Link: https://arxiv.org/abs/2105.11084


# Installations

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -q Archive.zip

In [3]:
!pip install git+https://github.com/pytorch/fairseq # need to install fairseq
!pip install -q python-levenshtein torchsummaryX wandb kaggle pytorch-nlp 
# You can install other libraries such as torchsummaryX, wandb and so on

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pytorch/fairseq to /tmp/pip-req-build-r2a3uy4i
  Running command git clone --filter=blob:none --quiet https://github.com/pytorch/fairseq /tmp/pip-req-build-r2a3uy4i
  Resolved https://github.com/pytorch/fairseq to commit 3f6ba43f07a6e9e2acf957fc24e57251a7a3f55c
  Running command git submodule update --init --recursive -q
  From https://github.com/ngoyal2707/Megatron-LM
   * branch            adb23324c222aad0aad89308e70302d996a5eaeb -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.6/269.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Kaggle

In [4]:
%pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle/

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"ksangam","key":"5ebf203383e0e2c1c9daeb15aa4d3283"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73272 sha256=0fb800159e13246a5f5283a77937b3e80155b4576a33582253e1a6faf90c7062
  Stored in directory: /root/.cache/pip/wheels/d4/02/ef/3f8c8d86b8d5388a1d3155876837f1a1a3143ab3fc2ff1ffad
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13


In [5]:
!kaggle competitions download -c 11-685-s23-hw5
!mkdir '/content/data'

!unzip -qo '/content/11-685-s23-hw5.zip' -d '/content/data'

100% 4.84G/4.85G [00:49<00:00, 81.7MB/s]
100% 4.85G/4.85G [00:49<00:00, 106MB/s] 


# Imports

In [6]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import *

import torchaudio.transforms as tat
import torchaudio.functional as Fa

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import Levenshtein
import sys
import json

has_cuda = torch.cuda.is_available()
if has_cuda:
  print("GPU: ", torch.cuda.get_device_name(0))
DEVICE = torch.device("cuda:0" if has_cuda else "cpu")
print("Device: ", DEVICE)

GPU:  Tesla T4
Device:  cuda:0


# Dataset and DataLoaders

You have TODOs which need to be completed in `task/unpaired_audio_text.py` before you run these cells. You just need to replace the paths. You can use the original code base as a reference.



In [7]:
from task import UnpairedAudioText

task = UnpairedAudioText()

In [8]:
train_dataloader_args = dict(batch_size=160, #feel free to change these values
                             num_workers=0,
                            ) if has_cuda else dict(batch_size=64)
train_dataloader_args["shuffle"] = True
train_dataloader_args["collate_fn"] = task.datasets["train"].collater

validation_dataloader_args = train_dataloader_args.copy()
validation_dataloader_args["shuffle"] = False
validation_dataloader_args["collate_fn"] = task.datasets["valid"].collater

train_dataloader = data.DataLoader(task.datasets["train"], **train_dataloader_args)
validation_dataloader = data.DataLoader(task.datasets["valid"], **validation_dataloader_args)

In [9]:
for data in train_dataloader:
    net_input = data['net_input']
    features = net_input['features'].to(DEVICE)
    padding_mask = net_input['padding_mask'].to(DEVICE)
    random_label = net_input['random_label'].to(DEVICE) if 'random_label' in net_input else None
    dense_x_only = net_input['dense_x_only'].to(DEVICE) if 'dense_x_only' in net_input else None
    segment = net_input['segment'].to(DEVICE) if 'segment' in net_input else None

    print(features.shape, padding_mask.shape)
    break

torch.Size([160, 261, 512]) torch.Size([160, 261])


# Model and Training Configurations

You need to complete the TODOs in `model/wav2vec_u.py` before you run this cell. You can use the original codebase as a refernce to complete this.
Original Codebase: https://github.com/pytorch/fairseq/blob/main/examples/wav2vec/unsupervised/


In [10]:
from model import Wav2vec_U

model = Wav2vec_U(task.target_dictionary).to(DEVICE)
print(model)

features, padding_mask = features.to(DEVICE), padding_mask.to(DEVICE)
summary(model, features, padding_mask, random_label, dense_x_only, segment)

Wav2vec_U(
  (discriminator): Discriminator(
    (net): Sequential(
      (0): Conv1d(64, 384, kernel_size=(6,), stride=(1,), padding=(5,))
      (1): SamePad()
      (2): Dropout(p=0.3, inplace=False)
      (3): Sequential(
        (0): Conv1d(384, 384, kernel_size=(6,), stride=(1,), padding=(5,))
        (1): SamePad()
        (2): Dropout(p=0.3, inplace=False)
        (3): GELU(approximate='none')
      )
      (4): Conv1d(384, 1, kernel_size=(6,), stride=(1,), padding=(5,))
      (5): SamePad()
    )
  )
  (segmenter): JoinSegmenter()
  (generator): Generator(
    (dropout): Dropout(p=0.1, inplace=False)
    (proj): Sequential(
      (0): TransposeLast()
      (1): Conv1d(512, 64, kernel_size=(4,), stride=(1,), padding=(2,), bias=False)
      (2): TransposeLast()
    )
  )
)
                                   Kernel Shape     Output Shape    Params  \
Layer                                                                        
0_generator.Dropout_dropout                   -  [160,

/usr/local/lib/python3.9/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_generator.Dropout_dropout,-,"[160, 261, 512]",NaN,NaN
1_generator.proj.TransposeLast_0,-,"[160, 512, 261]",NaN,NaN
2_generator.proj.Conv1d_1,"[512, 64, 4]","[160, 64, 262]",131072.0,34340864.0
3_generator.proj.TransposeLast_2,-,"[160, 262, 64]",NaN,NaN
4_discriminator.net.Conv1d_0,"[64, 384, 6]","[160, 384, 267]",147840.0,39370752.0
5_discriminator.net.SamePad_1,-,"[160, 384, 262]",NaN,NaN
6_discriminator.net.Dropout_2,-,"[160, 384, 262]",NaN,NaN
7_discriminator.net.3.Conv1d_0,"[384, 384, 6]","[160, 384, 267]",885120.0,236224512.0
8_discriminator.net.3.SamePad_1,-,"[160, 384, 262]",NaN,NaN


For a GAN, you need optimizers for both the discriminator and the generator. Configure the optimizers according to fairseq's configuration given in the link:
https://github.com/pytorch/fairseq/blob/main/examples/wav2vec/unsupervised/config/gan/w2vu.yaml


In [11]:
config = {
    'epochs': 3000
}

In [12]:
from itertools import chain

num_epochs = config['epochs']
epoch_start = 1

if epoch_start == 1:
    # define 2 optimizers for different parts of the model at the start of the training
    optimizer = {
      "discriminator": optim.Adam(
          model.discriminator.parameters(),
          # TODO: define lr, weight decay, betas and other relavant parameters
          lr=0.0005,
          betas=(0.5, 0.98),
          eps=1e-6,
          weight_decay=0.0001,
          amsgrad=False
        ),
      "generator": optim.Adam(
          chain(model.generator.parameters(), model.segmenter.parameters()),
          # TODO: define lr, weight decay, betas and other relavant parameters
          lr=0.0004,
          betas=(0.5, 0.98),
          eps=1e-6,
          weight_decay=0,
          amsgrad=False
        )
    }
    
# Optional TODO: Consider using mixed-precision to speed up training
scaler = torch.cuda.amp.GradScaler()

A bunch of TODOs in the next cell. <br><br>
Tip: Instead of completing whole `run_model` function and the debugging while running the experiment section, you can create a new cell and code your own sanity check. It may help you to understand what is returned from the dataloader, what needs to be pushed to the device, how model is called and what `loss_stats` are.

In [13]:
# Hint: You may find pdb to be a great tool in helping you understand returned values
# from the dataloader and the model. Usage:
# import pdb
# pdb.set_trace()

def run_model(model, dataloader):
    cumulative_stats = dict()

    for data in tqdm(dataloader, desc="Train" if model.training else "Eval"):

        for key in data['net_input'].keys():
            data['net_input'][key] = data['net_input'][key].to(DEVICE)

        if model.training:
            # TODO: We are training the model. Might need to do something with the optimizer?
            # Remember that you are training a GAN. Both optimizers won't be used at the same time.
            # You may have to write an if statement or something similar to use the specific optimizer.
            # You may have to use the discrim_step() attribute in the Wav2vec_U class
            group = model.get_groups_for_update() 
            optimizer[group].zero_grad()

            with torch.cuda.amp.autocast():
                # loss_stats = model(features, padding_mask, random_label, dense_x_only, segment) # forward pass
                loss_stats = model(**data['net_input'])
            
                total_loss = 0.0 

                # TODO: accumulate losses into total_loss for backprop during training
                # loss_stats["losses"] is a dictionary containing various loss components
                # some losses can be None if it's not used
                for loss in loss_stats['losses'].values():
                    if loss is not None:
                        total_loss += loss

                total_loss /= features.size(0) # average by batch

            # Look at what the get_groups_for_update() function does in the Wav2vec_U class
            # Can you try to think how you can use discrim_step() previously?
            # TODO: backprop loss and run the corresponding optimizer (Tip: See what 'group' is)
            scaler.scale(total_loss).backward()

            scaler.step(optimizer[group])
            scaler.update()

        else:
            # validation
            loss_stats = task.valid_step(data, model)


        # accumulate batch stats
        for k, v in loss_stats.items():
            if type(v) is dict:
                # flatten inner dictionary
                key_value_pairs = [(k + "_" + kn, vn) for kn, vn in v.items()]
            else:
                key_value_pairs = [(k, v)]

            # TODO: accmulate all statistics into cumulative_stats, a dictionary
            # NOTE: you should convert any returned tensors to either values or numpy arrays
            # cumulative_stats shouldn't have nested dictionaries
            for k, v in key_value_pairs:
                if v is not None:
                    if k in cumulative_stats:
                        cumulative_stats[k] += v
                    else:
                        cumulative_stats[k] = v

          
    # average stats over the dataset
    # Note that some metrics are already averaged over batch, so the result won't make sense
    # You can fix them if needed
    for k, v in cumulative_stats.items():
        v = v / len(dataloader.dataset)
        if type(v) is np.ndarray:
            v = v.tolist()
        cumulative_stats[k] = v

    return cumulative_stats

# WandB

In [14]:
import wandb
wandb.login(key="4a2e1698e528ee25e5c148a3dc6773bb92e5cfac")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [15]:
run = wandb.init(
    name = 'wav2vec-u', ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # id = 'drko6xof', ### Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw5", ### Project should be created in your wandb account 
    entity = 'idlhw5'
    # config = config ### Wandb Config for your run
)

wandb: Currently logged in as: ksangam (idlhw5). Use `wandb login --relogin` to force relogin


In [16]:
def save_model(model, optimizer_gen, optimizer_dis, epoch, path='./best.pt'):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_gen_state_dict'     : optimizer_gen.state_dict(),
         'optimizer_dis_state_dict'     : optimizer_dis.state_dict(),
         'epoch'                    : epoch}, 
         path
    )

def load_model(path, model, optimizer_gen, optimizer_dis):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    optimizer_gen.load_state_dict(checkpoint['optimizer_gen_state_dict'])
    optimizer_dis.load_state_dict(checkpoint['optimizer_dis_state_dict'])
        
    epoch   = checkpoint['epoch']

    return [model, optimizer_gen, optimizer_dis, epoch]

# Experiments

In [17]:
from datetime import datetime
now = datetime.now() # current date and time
dt = now.strftime("%m:%d_%H:%M:%S")
fn = 'w2v' + f'{dt}'
drive_dir = './drive/MyDrive/DeepLearning/h5/'
path = drive_dir + fn + '.pth'
print(path)

./drive/MyDrive/DeepLearning/h5/w2v04:25_12:16:37.pth


In [ ]:
print(f"Training for {num_epochs} epochs", file=sys.stderr)

best_dist = valid_dist = float("inf")
eval_interval = 10 # evaluation after how many epochs?

for epoch in range(epoch_start, num_epochs + 1):
    print(f"Epoch {epoch}", file=sys.stderr)

    model.train()
    # The model uses the epoch number to decide which part of the network to train
    model.set_num_updates(epoch) # Look at what this function does in the Wav2vec_U class

    train_stats = run_model(model, train_dataloader)

    if epoch % eval_interval == 0:

        model.eval()
        with torch.no_grad():
            eval_stats = run_model(model, validation_dataloader)
            
        # TODO: perhaps save your model and optimizer here
        # Tip: You can even save the model after every epoch along with the best model. 
        # This may help to continue training even if the best model is from a very early epoch.
        valid_dist = eval_stats['edit_distance']
        if valid_dist <= best_dist:
            best_dist = valid_dist
            # Save your model checkpoint here
            save_model(model, optimizer['generator'], optimizer['discriminator'], epoch, path)
            print("Saved model")

    else:
        eval_stats = {}

    # TODO: Log training/eval statistics
    # print("\tTrain Loss {:.04f}".format(train_stats['losses_smoothness']))

    logs = {}
    for key, val in train_stats.items():
        if isinstance(val, torch.Tensor):
            logs[key] = val.item()
        else:
            logs[key] = val
    if eval_stats:
        logs['edit_distance'] = eval_stats['edit_distance']
        logs['_num_pred_chars'] = eval_stats['_num_pred_chars']
    
    wandb.log(logs)
    
    if eval_stats:
        print("\tVal Dist {:.04f}\t".format(eval_stats['edit_distance']))

run.finish()

Training for 3000 epochs
Epoch 1
Train: 100%|██████████| 82/82 [00:37<00:00,  2.20it/s]
Epoch 2
Train: 100%|██████████| 82/82 [00:30<00:00,  2.69it/s]
Epoch 3
Train: 100%|██████████| 82/82 [00:32<00:00,  2.52it/s]
Epoch 4
Train: 100%|██████████| 82/82 [00:31<00:00,  2.61it/s]
Epoch 5
Train: 100%|██████████| 82/82 [00:31<00:00,  2.57it/s]
Epoch 6
Train: 100%|██████████| 82/82 [00:30<00:00,  2.69it/s]
Epoch 7
Train: 100%|██████████| 82/82 [00:31<00:00,  2.59it/s]
Epoch 8
Train: 100%|██████████| 82/82 [00:30<00:00,  2.71it/s]
Epoch 9
Train: 100%|██████████| 82/82 [00:31<00:00,  2.59it/s]
Epoch 10
Eval: 100%|██████████| 7/7 [00:02<00:00,  2.83it/s]
Epoch 11


Saved model
	Val Dist 114.8190	


Train: 100%|██████████| 82/82 [00:31<00:00,  2.59it/s]
Epoch 12
Train: 100%|██████████| 82/82 [00:30<00:00,  2.71it/s]
Epoch 13
Train: 100%|██████████| 82/82 [00:31<00:00,  2.58it/s]
Epoch 14
Train: 100%|██████████| 82/82 [00:29<00:00,  2.74it/s]
Epoch 15
Train: 100%|██████████| 82/82 [00:31<00:00,  2.60it/s]
Epoch 16
Train: 100%|██████████| 82/82 [00:29<00:00,  2.76it/s]
Epoch 17
Train: 100%|██████████| 82/82 [00:31<00:00,  2.60it/s]
Epoch 18
Train: 100%|██████████| 82/82 [00:29<00:00,  2.75it/s]
Epoch 19
Train: 100%|██████████| 82/82 [00:31<00:00,  2.59it/s]
Epoch 20
Eval: 100%|██████████| 7/7 [00:02<00:00,  2.52it/s]
Epoch 21


Saved model
	Val Dist 111.1500	


Train: 100%|██████████| 82/82 [00:31<00:00,  2.63it/s]
Epoch 22
Train: 100%|██████████| 82/82 [00:29<00:00,  2.75it/s]
Epoch 23
Train: 100%|██████████| 82/82 [00:31<00:00,  2.59it/s]
Epoch 24
Train: 100%|██████████| 82/82 [00:30<00:00,  2.73it/s]
Epoch 25
Train: 100%|██████████| 82/82 [00:31<00:00,  2.58it/s]
Epoch 26
Train: 100%|██████████| 82/82 [00:29<00:00,  2.76it/s]
Epoch 27
Train: 100%|██████████| 82/82 [00:32<00:00,  2.53it/s]
Epoch 28
Train: 100%|██████████| 82/82 [00:29<00:00,  2.81it/s]
Epoch 29
Train: 100%|██████████| 82/82 [00:32<00:00,  2.52it/s]
Epoch 30
Eval: 100%|██████████| 7/7 [00:03<00:00,  2.03it/s]
Epoch 31


...

Train: 100%|██████████| 82/82 [00:32<00:00,  2.52it/s]
Epoch 2672
Train: 100%|██████████| 82/82 [00:27<00:00,  2.95it/s]
Epoch 2673
Train: 100%|██████████| 82/82 [00:31<00:00,  2.58it/s]
Epoch 2674
Train: 100%|██████████| 82/82 [00:27<00:00,  2.96it/s]
Epoch 2675
Train: 100%|██████████| 82/82 [00:32<00:00,  2.51it/s]
Epoch 2676
Train: 100%|██████████| 82/82 [00:27<00:00,  2.97it/s]
Epoch 2677
Train: 100%|██████████| 82/82 [00:31<00:00,  2.57it/s]
Epoch 2678
Train: 100%|██████████| 82/82 [00:27<00:00,  2.97it/s]
Epoch 2679
Train: 100%|██████████| 82/82 [00:31<00:00,  2.57it/s]
Epoch 2680
Eval: 100%|██████████| 7/7 [00:02<00:00,  2.64it/s]
Epoch 2681


	Val Dist 55.9000	


Train: 100%|██████████| 82/82 [00:31<00:00,  2.63it/s]
Epoch 2682
Train: 100%|██████████| 82/82 [00:27<00:00,  2.95it/s]
Epoch 2683
Train: 100%|██████████| 82/82 [00:30<00:00,  2.65it/s]
Epoch 2684
Train: 100%|██████████| 82/82 [00:28<00:00,  2.88it/s]
Epoch 2685
Train: 100%|██████████| 82/82 [00:30<00:00,  2.65it/s]
Epoch 2686
Train: 100%|██████████| 82/82 [00:27<00:00,  2.94it/s]
Epoch 2687
Train: 100%|██████████| 82/82 [00:31<00:00,  2.63it/s]
Epoch 2688
Train: 100%|██████████| 82/82 [00:28<00:00,  2.83it/s]
Epoch 2689
Train: 100%|██████████| 82/82 [00:31<00:00,  2.59it/s]
Epoch 2690
Eval: 100%|██████████| 7/7 [00:02<00:00,  3.17it/s]
Epoch 2691


	Val Dist 68.1600	


Train: 100%|██████████| 82/82 [00:31<00:00,  2.60it/s]
Epoch 2692
Train: 100%|██████████| 82/82 [00:28<00:00,  2.87it/s]
Epoch 2693
Train: 100%|██████████| 82/82 [00:31<00:00,  2.60it/s]
Epoch 2694
Train: 100%|██████████| 82/82 [00:27<00:00,  2.98it/s]
Epoch 2695
Train: 100%|██████████| 82/82 [00:31<00:00,  2.58it/s]
Epoch 2696
Train: 100%|██████████| 82/82 [00:28<00:00,  2.83it/s]
Epoch 2697
Train:   1%|          | 1/82 [00:00<00:58,  1.39it/s]

In [21]:
run.finish()

code_ppl,▄█▆█▄▄▄▃▁▃▃▄▄▃▄▂▃▃▅▃▃▂▂▄▃▄▁▅▃▄▃▅▃▆▅▄▃▄▄▄
d_steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
edit_distance,▁▆▄▆█▄▅▄▅▅▅▆▅▅▆▇▄▄▃▅▅▅▃▃▄▄▄▅▆▄▇▄▅▅▄▅▄▅▅▆
losses_code_pen,▁▄▆▅▇▇▆▇█▇▇▇▇▇▇█▇▇▆▇▇█▇▇▇▇█▆▇▇▇▆▇▆▆▆▇▇▇▇
losses_dense_dis,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses_dense_gen,▂▁▂▁▃▃▄▃▅▄▄▄▅▄▃▆▄▅▄▄▅▄▄▄▅▄▃▄▄█▇▄▇▅▃▅▅▅█▅
losses_grad_pen,█▃▂▂▂▂▂▂▂▂▂▁▁▂▁▂▂▂▂▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▂
losses_smoothness,▁▄▅▅▆▆▇▆▇▇▇▇▇▇▇█▇▇▇▆▇▇▇▇▇▇▆▇▇█▇▇██▇▇▇▇██
losses_token_dis,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
prob_ppl,█▅▃▄▂▂▃▂▁▂▂▂▂▂▂▁▂▂▃▂▂▁▂▂▂▂▁▃▂▂▂▃▂▃▃▃▂▂▂▂
sample_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [ ]:
save_model(model, optimizer, None, None, epoch, path, scaler)
run.finish()

code_ppl,▆█▅▄▄▄▃▅▂▄▃▂▄▂▂▂▃▂▁▃▃▄▂▃▂▂▂▁▂▂▁▃▂▁▁▃▂▂▃▁
d_steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
edit_distance,█▆▅▅▄▆▆▅▅▄▅▄▆▅▄▃▁▅▄▅▆▅▄▄▄▄▃▅▆▆▅▃▆▄▅▄▅▆▅▄
losses_code_pen,▁▂▅▅▆▅▆▅▇▅▇▇▆▇█▇▆██▆▇▆▇▆▇▇▇█▇▇█▆▇██▇▇▇▆█
losses_dense_dis,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses_dense_gen,▁▂▂▂▃▃▃▄▄▃▄▄▅▄▅▅▄▆▅▄▅▆▅▅▅▇▆▅▅▅▅▅▅▅▇▅█▆▅█
losses_grad_pen,█▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▂▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▂▂
losses_smoothness,▁▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▆▇▇▇▆▇▇▇▇█▇▇▇▇▆▇▆▇▇▇▇▇▇█
losses_token_dis,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
nsentences,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
prob_ppl,█▇▄▄▃▄▃▄▂▄▂▂▃▂▁▂▃▁▁▃▂▃▂▃▂▂▂▁▂▂▁▃▂▁▁▂▂▂▃▁


# Testing and submission to Kaggle

In [ ]:
from dataset import extracted_features_dataset
test_path = # TODO: PathToTest
testset = extracted_features_dataset.ExtractedFeaturesDataset(path=test_path,
                                                              split='test')

test_dataloader_args = train_dataloader_args.copy()
test_dataloader_args["shuffle"] = False
test_dataloader_args["collate_fn"] = testset.collater

test_dataloader = data.DataLoader(testset, **test_dataloader_args)

Write the `test_step` function which can be coded very similar to `valid_step` given in `task/unpaired_audio_text.py`

In [ ]:
def test_step(inputs, model):
    # TODO
    pass

Write some code to evaluate and get the results. You are free to write the below cells however you want

In [ ]:
model.eval()
results = []

for data in tqdm(test_dataloader, desc="Test"):
    # TODO
    pass

In [ ]:
# TODO: Replace the path and get the phoneme_map.json for mapping
with open("/Path/To/phoneme_map.json", "r") as file:
    phon_map = json.load(file)

In [ ]:
predictions = []

for line in results:
    # TODO: Map results with phon_map
    pass

In [ ]:
# TODO: Make the CSV and submit to kaggle